# Decision probabilities in committees with costly expert information
This notebook is designed to illustrate the probability a committee privy to both freely available private information, as well as costly information that a majority of agents must contribute to acquiring, will make the correct decision between two alternatives in a majority vote. See pages 3-4 in my [dissertation](https://warwick.ac.uk/fac/soc/economics/research/wmesp/manage/61_-_jonathan_newman.pdf) for an outline of the set-up and preliminaries.

*This code was written by Jonathan Newman (jonathannewman55@gmail.com)*

### Set up Ipython environment

Jupyter notebook runs in an iPython environment meaning you will need iPython to autoreload imported files (especially custom packages)

In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load appropriate modules and packages

In [12]:
import numpy as np
import pandas as pd
import math
from scipy.stats import binom
import sys
import scipy

### Preliminaries
First let's define the utility gain to each agent from acquiring the costly expert information (excluding contributions):
- $\beta(n,p,q) = q - \sum_{j=\frac{n+1}{2}}^{n}b(j;n,p)$

And the maximum cost that will support a mixed equilibrium where a positive number of agents contribute:
- $c^{\text{max}}=\binom{n-1}{\frac{n-1}{2}}\frac{1}{2}^{n-1}\beta(n,p,q)$

In [3]:
## Benefit of acquiring expert signal
def beta(n,p,q):
    b = 1-binom.cdf((n-1)/2,n,p)
    beta = q - b
    return beta

## max supporting cost for contributing equilibria
def c_max(n,p,q):
    fact = math.comb(int((n-1)),int((n-1)/2))
    c_max = fact*0.5**(n-1)*beta(n,p,q)
    return c_max


### Contribution and acquisition probabilities
Next, set out the contribution $\psi$ and acquisition $\Psi$ probabilities which will feed into the decision probability $\Phi$.

$\psi$ is defined as a function of the probability $i$ is pivotal to acquiring the expert signal, itself obtained by setting $i$'s expected payoff from contributing equal to their expected payoff from not contributing:

- $\text{Expected benefit of contributing} = \text{expected benefit of abstaining} \iff \frac{c}{\beta(n,p,q)}=b(\frac{n-1}{2}; n-1,\psi)$

And the acquisition probability $\Psi$ is the probability a majority ($\frac{n+1}{2}$) of agents contribute to acquiring the expert signal:

- $\Psi=\sum_{j=\frac{n+1}{2}}^{n}b(j;n,\psi)$

In [4]:
## acute contribution probability
def psi(n,p,q,c):
    ## agents will never contribute if cost too high
    if c > c_max(n,p,q):
        return 0
    else:
        ## compute the probability i is pivotal to acquiring s_{e}
        pivot = (c/beta(n,p,q))
        fact = math.comb(int((n-1)),int((n-1)/2))
        intercept = (pivot/fact)**(2/(n-1))

        ## after some rearranging we arrive at a quadratic in \psi
        psi_acute = (1 + math.sqrt(1-4*intercept))/2
        return psi_acute
    
## acute acquisition probability
def acq(n,p,q,c):
    psi_acute = psi(n,p,q,c)
    ## formula invariant of equilibrium probabilities
    acq_prob = 1-binom.cdf((n-1)/2,n,psi_acute)
    return acq_prob

### The decision probability
The decision probability is finally computed thus:

- $\Phi=\Psi q+[1-\Psi]\sum_{j=\frac{n+1}{2}}^{n}b(j;n,p)$

With $q$ being the probability the committee makes the correct decision when agents acquire the expert signal and $\sum_{j=\frac{n+1}{2}}^{n}b(j;n,p)$ the benefit of agglomerating everyone's private signals when the committee doesn't acquire the expert signal.

In [5]:
## designed as a wrap-around for the previous user functions
from collections import UserList
## acute decision probability
def dec(n,p,q,c):
    ## make all numerical inputs for "n" a numpy array for ease
    if not isinstance(n,np.ndarray):
        n = np.array(n)
    ## force n to be a 1-dimensional array
    n = np.atleast_1d(n)
    dec_prob = []
    for size in n:
        ## check if each n is an odd integer as per the model
        if size % 2 == 1:
            acq_acute = acq(size,p,q,c)
            prob = q*acq_acute + (1-binom.cdf((size-1)/2,size,p))*(1-acq_acute)
            dec_prob.append(prob)
        else: 
            raise ValueError("N should be an odd integer or an array of odd integers")
    return dec_prob

We can then chart $\Phi$ as a function of committee size for a range of cost inputs, using Plotly's slider functionality.

In [13]:
## And now plot those decision probabilities for a range of costs
import plotly.graph_objects as go
plot = go.Figure()
## Fix signal qualities
p=0.55
q=0.95
## n extends through the odd numbers to 71
n=np.arange(3,71,2)
## cost parameter to range between 0 and 0.1
for cost in np.arange(0.001,0.1,0.001):
    ## add hidden traces of scatter plots for each step in the cost function
    plot.add_trace(
        go.Scatter(visible=False,line=dict(width=2),
                   name="cost = " + str(cost),x=n,y=dec(n,p,q,cost)))

In [14]:
## build slider to demonstrate probabilities for a range of costs in different panels
## see https://plotly.com/python/sliders/ from which this section of code is adapted
cost_array = []
for j in range(len(plot.data)):
    cost = dict(
        method="restyle",args=[{"visible": ([False] * len(plot.data))}],
        label=np.arange(0.001,0.1,0.001)[j])
    ## match the slider position with the visible frame
    cost["args"][0]["visible"][j] = True
    cost_array.append(cost)
slide = [dict(currentvalue=dict(visible=False),pad=dict(t=50),steps=cost_array)]

In [15]:
## get plot to initially show the graph of c=0.001
plot.data[0].visible = True
## finally adjust title/axes etc.
plot.update_layout(sliders=slide)
text = f"Plot of the Decision Probability \u03D5(n) for p={p},q={q} and c \u2208 (0,0.1]"
plot.update_layout(title={'text': text,'y':0.9,'x':0.5})
plot.update_layout(xaxis_title="Number of agents n", yaxis_title="\u03D5(n)")
## get white background and re-size image
plot.update_layout(width = 960, height = 640,plot_bgcolor ='#ffffff')

plot.show()